In [4]:
import tensorflow as tf

In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [6]:
df = pd.read_csv('solution.csv')

In [7]:
df.head()

,id,category
0,1,5
1,2,4
2,3,6
3,4,2
4,5,2


In [8]:
dummies=[]
cols = ['category']
for col in cols:
    dummies.append(pd.get_dummies(df[col]))
dummie_df = pd.concat(dummies,axis=1)
df = pd.concat((df,dummie_df),axis=1)

In [9]:
df.head()

,id,category,1,2,3,4,5,6
0,1,5,0,0,0,0,1,0
1,2,4,0,0,0,1,0,0
2,3,6,0,0,0,0,0,1
3,4,2,0,1,0,0,0,0
4,5,2,0,1,0,0,0,0


In [10]:
df.drop(['category'],axis=1,inplace=True)

In [11]:
df.head()

,id,1,2,3,4,5,6
0,1,0,0,0,0,1,0
1,2,0,0,0,1,0,0
2,3,0,0,0,0,0,1
3,4,0,1,0,0,0,0
4,5,0,1,0,0,0,0


In [12]:
cat_df = pd.read_csv('solution.csv')

In [13]:
mode = cat_df['category'].mode()

In [14]:
mode

0    4
dtype: int64

In [15]:
import shutil
import glob

In [58]:
import os
#TODO:-images ko apni category wale folder me dalna hai
def load_images_from_folder(folder):
    images = []
#     for filename in os.listdir(folder)[:20]:
#         img = filename.split('.')[0]
#         int_id = int(img)
#         file = glob.glob('./training/filename')
#         new_folder = './' + str(cat_df.loc[[int_id]].category)
#         shutil.copy(file[0], new_folder)
for jpgfile in glob.iglob(os.path.join('./training', "*.png")):
    img_id = int(jpgfile.split('/')[2][:-4])
    loc = int(cat_df.iloc[[img_id-1]]['category'])
    shutil.copy(jpgfile, './' + str(loc))

In [35]:
# img_data = load_images_from_folder('./training')

In [64]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 6, activation = 'sigmoid'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [65]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('./dataset',
target_size = (64, 64),
batch_size = 32,
classes = ['1','2','3','4','5','6'],    # yaha dekho                                             
class_mode = 'categorical')             # or yaha bhi

Found 4616 images belonging to 6 classes.


In [66]:
test_set = test_datagen.flow_from_directory('./testset',
target_size = (64, 64),
batch_size = 32,
class_mode = 'categorical')

Found 384 images belonging to 6 classes.


In [ ]:
classifier.fit_generator(training_set,
steps_per_epoch = 8000,
epochs = 1,
validation_data = test_set,
validation_steps = 2000)

Epoch 1/25
8000/8000 [==============================] - 2614s 327ms/step - loss: 0.3441 - acc: 0.8411 - val_loss: 0.3628 - val_acc: 0.8199
Epoch 2/25
  13/8000 [..............................] - ETA: 1:02:53 - loss: 0.2762 - acc: 0.8670